# Run autoaugment to obtain best parameters
       -- built for FF+ dataset with file structure as required by Keras' flow_from_directory method
       
Requires 50GB RAM, at least 4 CPU workers and a GPU.

In [ ]:
# See available GPU RAM 
# !nvidia-smi # can also be run from linux shell while GPU is training
# !nvidia-smi dmon # this will stream memory utilisation
# !watch -n0.1 nvidia-smi # better way to see GPU utilisation
# !htop # cpu threads and if they're all working
# !pip3 install --user --no-cache-dir -I tensorflow==2.2 # use if no gpu is attached so code will run 
# !sudo kill -9 PID # clear GPU memory where 9 is PID number
# !sudo sh -c 'echo 3 > /proc/sys/vm/drop_caches' # clear CPU memory

In [1]:
# Autoaugment installations and run TF2 upgrade scripts
# !tf_upgrade_v2 --intree ../augmentations/deepaugment-master --outtree ../augmentations/deepaugment_updated
# Then make some manual changes to ensure it runs using TF2
# Now install packages required by DeepAugment
# !pip install scikit-optimize
# !pip install imgaug

     |████████████████████████████████| 80 kB 3.6 MB/s eta 0:00:011
     |████████████████████████████████| 948 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 43.7 MB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 81.8 MB/s eta 0:00:01


In [1]:
# This cell has the latest set up for AI Platform
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import glob
import cv2
from io import BytesIO
from PIL import Image
from numpy import expand_dims
import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import *
import warnings
import logging
from IPython.display import clear_output
from collections import Counter
import pickle
import json
import sys
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/augmentations')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/augmentations/deepaugment_updated/deepaugment')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/hyperparameters')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/visualisations')
import VisualisationTools as plotting
import hyper_utils as hp
import deepaugment as dp

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(100)
plot_losses = plotting.PlotLearning()
os.chdir('/home/jupyter/DeepFake-2019-20/augmentations')

# # Augmentation libraries
# import face_recognition
# import cutout_augmentation as ca

print("Tensorflow version:", tf.__version__)

Using TensorFlow backend.



Tensorflow version: 2.1.1-dlenv_tfe


## Load images into memory

In [2]:
train_authentic_image_paths = os.listdir('../../all_faces_disk/home/jupyter/forensics_split/train/authentic')
train_authentic_images = []

# Training data
for image_path in train_authentic_image_paths:
    img = cv2.imread('../../all_faces_disk/home/jupyter/forensics_split/train/authentic/' + image_path)
    # resizing is optional
    img=cv2.resize(img,(224,224))
    train_authentic_images.append(img)
    if len(train_authentic_images) % 1000 == 0: print(len(train_authentic_images))
        
    if len(train_authentic_images) == 2510:
        break
        
        
train_fake_image_paths = os.listdir('../../all_faces_disk/home/jupyter/forensics_split/train/fake')
train_fake_images = []

for image_path in train_fake_image_paths:
    img = cv2.imread('../../all_faces_disk/home/jupyter/forensics_split/train/fake/' + image_path)
    # resizing is optional
    img=cv2.resize(img,(224,224))
    train_fake_images.append(img)
    if len(train_fake_images) % 1000 == 0: print(len(train_fake_images))
        
    if len(train_fake_images) == 2510:
        break

1000
2000
1000
2000


In [3]:
# Concatenate authentic and fake images
training_data = np.array(train_authentic_images + train_fake_images)

# Create training labels
train_labels = np.array([0]*len(train_authentic_images) + [1]*len(train_fake_images))

print(training_data.shape, train_labels.shape)

(5020, 224, 224, 3) (5020,)


## Run Deepaugment

In [4]:
# Run DeepAugment

config = {
    "model": "basiccnn",
    "method": "bayesian_optimization",
    "train_set_size": 5000,
    "opt_samples": 1,
    "opt_last_n_epochs": 3,
    "opt_initial_points": 10,
    "child_epochs": 20,
    "child_first_train_epochs": 0,
    "child_batch_size": 64,
    "notebook_path": "./autoaugment_results.csv"
}

# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
deepaug = dp.DeepAugment(training_data, train_labels, config=config)

Using 5000 training images
BasicCNN model built as child model.
 Model summary:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 32)      0         
__________________________________________

In [ ]:
best_policies = deepaug.optimize(100)

trial: 1 
 ['gamma-contrast', 0.8442657485810175, 'coarse-salt-pepper', 0.8472517387841256, 'brighten', 0.38438170729269994, 'translate-y', 0.056712977317443194, 'translate-y', 0.47766511732135, 'add-to-hue-and-saturation', 0.47997717237505744, 'emboss', 0.8360787635373778, 'sharpen', 0.6481718720511973, 'emboss', 0.9571551589530466, 'rotate', 0.8700872583584366]
gamma-contrast
brighten
translate-y
emboss
emboss
load_pre_augment_weights()'s runtime:  0.5529 sec.
Train on 30000 samples, validate on 20 samples
Epoch 1/20
30000/30000 - 131s - loss: 0.8673 - accuracy: 0.4990 - val_loss: 0.6875 - val_accuracy: 0.7500
Epoch 2/20
30000/30000 - 129s - loss: 0.6956 - accuracy: 0.5060 - val_loss: 0.6919 - val_accuracy: 0.7000
Epoch 3/20
30000/30000 - 128s - loss: 0.6948 - accuracy: 0.5248 - val_loss: 0.7007 - val_accuracy: 0.3000
Epoch 4/20
30000/30000 - 128s - loss: 0.6873 - accuracy: 0.5473 - val_loss: 0.7062 - val_accuracy: 0.3000
Epoch 5/20
30000/30000 - 128s - loss: 0.6828 - accuracy: 0.559

## Obtain best policies 

In [ ]:
def get_top_policies(df, k):
        """Prints and returns top-k policies

        Policies are ordered by their expected accuracy increas
        Args:
            k (int) top-k
        Returns
            pandas.DataFrame: top-k policies as dataframe
        """
        trial_avg_val_acc_df = (
            df.drop_duplicates(["trial_no", "sample_no"])
            .groupby("trial_no")
            .mean()["mean_late_val_acc"]
            .reset_index()
        )[["trial_no", "mean_late_val_acc"]]

        x_df = pd.merge(
            df.drop(columns=["mean_late_val_acc"]),
            trial_avg_val_acc_df,
            on="trial_no",
            how="left",
        )

        x_df = x_df.sort_values("mean_late_val_acc", ascending=False)

        baseline_val_acc = x_df[x_df["trial_no"] == 0]["mean_late_val_acc"].values[0]

        x_df["expected_accuracy_increase(%)"] = (
            x_df["mean_late_val_acc"] - baseline_val_acc
        )*100

        top_df = x_df.drop_duplicates(["trial_no"]).sort_values(
            "mean_late_val_acc", ascending=False
        )[:k]

        SELECT = [
            "trial_no",
            'A_aug1_type', 'A_aug1_magnitude', 'A_aug2_type', 'A_aug2_magnitude',
            'B_aug1_type', 'B_aug1_magnitude', 'B_aug2_type', 'B_aug2_magnitude',
            'C_aug1_type', 'C_aug1_magnitude', 'C_aug2_type', 'C_aug2_magnitude',
            'D_aug1_type', 'D_aug1_magnitude', 'D_aug2_type', 'D_aug2_magnitude',
            'E_aug1_type', 'E_aug1_magnitude', 'E_aug2_type', 'E_aug2_magnitude',
            "mean_late_val_acc", "expected_accuracy_increase(%)"
        ]
        top_df = top_df[SELECT]

        print(f"top-{k} policies:", k)
        print(top_df)

        return top_df

get_top_policies(pd.read_csv('./autoaugment_results.csv'), 20)